In [1]:
from dask import dataframe as dd
from datetime import datetime

In [2]:
def parse_datetime(dt):
    date_format = '%Y-%m-%d %H:%M:%S.%f'
    try:
        dt = datetime.strptime(dt[:-4], date_format).timestamp()
    except ValueError:
        dt = datetime.strptime(dt[:-4], date_format[:-3]).timestamp()

    dt *= 1000.0
    dt = int(dt)
    dt -= 1648806250315

    return dt

In [2]:
def parse_timestamp(timestamp):
    date_format = '%Y-%m-%d %H'

    timestamp /= 1000
    timestamp = int(timestamp)
    timestamp += 1648806250

    dt = datetime.fromtimestamp(timestamp).strftime(date_format)
    dt = datetime.strptime(dt, date_format)

    return dt

In [4]:
def parse_color(pixel_color):

    hex_to_key = {
        "#000000": 0,
        "#00756F": 1,
        "#009EAA": 2,
        "#00A368": 3,
        "#00CC78": 4,
        "#00CCC0": 5,
        "#2450A4": 6,
        "#3690EA": 7,
        "#493AC1": 8,
        "#515252": 9,
        "#51E9F4": 10,
        "#6A5CFF": 11,
        "#6D001A": 12,
        "#6D482F": 13,
        "#7EED56": 14,
        "#811E9F": 15,
        "#898D90": 16,
        "#94B3FF": 17,
        "#9C6926": 18,
        "#B44AC0": 19,
        "#BE0039": 20,
        "#D4D7D9": 21,
        "#DE107F": 22,
        "#E4ABFF": 23,
        "#FF3881": 24,
        "#FF4500": 25,
        "#FF99AA": 26,
        "#FFA800": 27,
        "#FFB470": 28,
        "#FFD635": 29,
        "#FFF8B8": 30,
        "#FFFFFF": 31,
    }

    if type(pixel_color) is str:
        return hex_to_key[pixel_color]
    else:
        return [key for key, value in hex_to_key.items() if value == pixel_color][0]


## Оптимизиране на оригиналната информация. Премахване на колоната user_id

In [25]:
ddf_core = dd.read_csv('data\\2022_place_canvas_history.csv', converters={'timestamp': parse_datetime, 'pixel_color': parse_color})

ddf_core = ddf_core.drop('user_id', axis=1)
ddf_core.astype({'timestamp': 'uint32'}).dtypes
ddf_core.astype({'timestamp': 'uint8'}).dtypes
ddf_core = ddf_core.sort_values('timestamp')

ddf_core.head()

,timestamp,pixel_color,coordinate
97731,0,14,"42,42"
97732,12356,3,"999,999"
97733,16311,7,"44,42"
97734,21388,21,"2,2"
97735,34094,7,"23,23"


In [26]:
ddf_core.to_csv('data\\2022_place_canvas_history_core.csv', index=False, single_file=True)

['f:\\Дипломна работа\\code\\data\\raw\\2022_place_canvas_history_core.csv']

## Експортиране на колоната user_id в отделен файл

In [27]:
ddf_users = dd.read_csv('data\\2022_place_canvas_history.csv')

ddf_users = ddf_users.drop(['timestamp','pixel_color','coordinate'],axis=1)

ddf_users.head()

,user_id
0,ovTZk4GyTS1mDQnTbV+vDOCu1f+u6w+CkIZ6445vD4XN8a...
1,6NSgFa1CvIPly1VniNhlbrmoN3vgDFbMSKqh+c4TTfrr3d...
2,O5Oityp3Z3owzTuwM9XnMggpLcqKEumsOMKGhRiDTTImWb...
3,tc273UiqS0wKa6VwiOs/iz/t4LyPYrhL2Q347awn11IQQE...
4,OOWsU/HLb4UUkQwclDeXFtsJTOXMlAdNHiRpFA1Qk+SxUr...


In [28]:
ddf_users.to_csv('data\\2022_place_canvas_history_users.csv', index=False, single_file=True)

['f:\\Дипломна работа\\code\\data\\raw\\2022_place_canvas_history_users.csv']

## Завъртане на pixel_color колоната и преброяване на видовете и броя цветове за всеки час

In [5]:
ddf_pivot = dd.read_csv('data\\2022_place_canvas_history_core.csv')
ddf_pivot['timestamp'] = ddf_pivot['timestamp'].map(parse_timestamp)
ddf_pivot['pixel_color'] = ddf_pivot['pixel_color'].map(parse_color)

ddf_pivot = ddf_pivot.drop('coordinate', axis=1)
ddf_pivot = ddf_pivot.rename(columns={'timestamp': 'datetime'})
ddf_pivot['index'] = ddf_pivot.index

ddf_pivot.head()

,datetime,pixel_color,index
0,2022-04-01 12:00:00,#7EED56,0
1,2022-04-01 12:00:00,#00A368,1
2,2022-04-01 12:00:00,#3690EA,2
3,2022-04-01 12:00:00,#D4D7D9,3
4,2022-04-01 12:00:00,#3690EA,4


In [6]:
ddf_pivot['pixel_color'] = ddf_pivot['pixel_color'].astype('category')
ddf_pivot['pixel_color'] = ddf_pivot.pixel_color.cat.as_known()

ddf_pivot.dtypes

datetime       datetime64[ns]
pixel_color          category
index                   int64
dtype: object

In [7]:
ddf_pivot = ddf_pivot.pivot_table(index='datetime', columns='pixel_color', values='pixel_color', aggfunc='count')

ddf_pivot.head()

pixel_color,#000000,#00A368,#2450A4,#3690EA,#51E9F4,#7EED56,#811E9F,#898D90,#9C6926,#B44AC0,...,#BE0039,#FF3881,#00CCC0,#515252,#6D001A,#94B3FF,#DE107F,#E4ABFF,#FFB470,#FFF8B8
datetime,,,,,,,,,,,,,,,,,,,,,
2022-04-01 12:00:00,0.0,2.0,3.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-01 13:00:00,117511.0,21775.0,83172.0,15056.0,24328.0,21971.0,19421.0,7081.0,6169.0,5815.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-01 14:00:00,198737.0,33085.0,115221.0,20169.0,38124.0,30806.0,26811.0,12597.0,9609.0,7658.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-01 15:00:00,242980.0,35452.0,98686.0,25440.0,42816.0,33479.0,27869.0,17086.0,12235.0,10745.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-01 16:00:00,246588.0,40658.0,99270.0,29857.0,49147.0,37166.0,32077.0,20787.0,11657.0,9305.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
columns = ddf_pivot.columns
for column in columns:
    ddf_pivot[column] = ddf_pivot[column].astype('uint32')

ddf_pivot.dtypes

pixel_color
#000000    uint32
#00A368    uint32
#2450A4    uint32
#3690EA    uint32
#51E9F4    uint32
#7EED56    uint32
#811E9F    uint32
#898D90    uint32
#9C6926    uint32
#B44AC0    uint32
#D4D7D9    uint32
#FF4500    uint32
#FF99AA    uint32
#FFA800    uint32
#FFD635    uint32
#FFFFFF    uint32
#00756F    uint32
#009EAA    uint32
#00CC78    uint32
#493AC1    uint32
#6A5CFF    uint32
#6D482F    uint32
#BE0039    uint32
#FF3881    uint32
#00CCC0    uint32
#515252    uint32
#6D001A    uint32
#94B3FF    uint32
#DE107F    uint32
#E4ABFF    uint32
#FFB470    uint32
#FFF8B8    uint32
dtype: object

In [9]:
ddf_pivot.to_csv('data\\2022_place_canvas_history_pivoted.csv', single_file=True)

['f:\\Дипломна работа\\code\\data\\raw\\2022_place_canvas_history_pivoted.csv']

## Изнасяне на колоната datetime в отделен файл + брой пиксели поставени за всеки час

In [3]:
ddf_date = dd.read_csv('data\\2022_place_canvas_history_core.csv')
ddf_date = ddf_date.drop({'pixel_color', 'coordinate'}, axis=1)
ddf_date = ddf_date.rename(columns={'timestamp': 'datetime'})
ddf_date['datetime'] = ddf_date['datetime'].map(parse_timestamp)

ddf_date.head()

,datetime
0,2022-04-01 12:00:00
1,2022-04-01 12:00:00
2,2022-04-01 12:00:00
3,2022-04-01 12:00:00
4,2022-04-01 12:00:00


In [4]:
ddf_date['datetime'] = dd.to_datetime(ddf_date['datetime'])
ddf_date.dtypes

datetime    datetime64[ns]
dtype: object

In [5]:
ddf_date = ddf_date.datetime.value_counts('datetime').compute()
ddf_date.head()

2022-04-04 21:00:00    5901646
2022-04-04 22:00:00    5640455
2022-04-04 20:00:00    4904479
2022-04-04 19:00:00    4567933
2022-04-03 21:00:00    4480088
Name: datetime, dtype: int64

In [11]:
ddf_date = ddf_date.reset_index(name='num_pixels').sort_values('index').set_index('index')
ddf_date = ddf_date.reset_index()
ddf_date = ddf_date.rename(columns={'index': 'datetime'})
ddf_date.head()

,datetime,num_pixels
0,2022-04-01 12:00:00,19
1,2022-04-01 13:00:00,548836
2,2022-04-01 14:00:00,867847
3,2022-04-01 15:00:00,996924
4,2022-04-01 16:00:00,1070919


In [8]:
ddf_date.to_csv('data\\2022_place_canvas_history_hourly-pixels.csv')